In [82]:
from selenium import webdriver
from datetime import datetime
from time import sleep
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
import shutil
import time

# urllib库

## 网页代码urlopen，urlretrieve

### request.urlopen( ) 获取网页代码

In [8]:
from urllib import request
resp=request.urlopen('http://www.baidu.com/',data=None)     #指定data，urlopen走的是post请求，默认情况下走的是get请求.
resp.read()                    #resp是一个http.client.HTTPResponse对象 
                               #这个返回对象有read(),readline(),readlines(),getcode()等方法。返回byte数据

b'<!DOCTYPE html><!--STATUS OK-->\n'

### request.urlretrieve( ) 下载网页代码

In [11]:
from urllib import request
request.urlretrieve('http://www.baidu.com/','baidu.html') #直接下载+储存html文件，用来下载多张图片很方便（直接获取图片url）

('baidu.html', <http.client.HTTPMessage at 0x10a8e86d8>)

### ProxyHandler处理器 IP代理

In [ ]:
from urllib import request
url='http://httpbin.org/ip'                                   #查看当前ip地址的网页

#这个是没有用代理的，真实的ip
resp = request.urlopen(url)               
print(resp.read().decode("utf-8"))

#代理
handler= request.ProxyHandler({"http":"120.83.104.208:9999"}) #创建一个代理handler，“ip地址：port端口” kuaidaili.com
opener=request.build_opener(handler)                          #使用handler创建opener   #(不要用免费代理）
resp=opener.open(url)                                         #使用opener发送请求 opener.open(url)
print(resp.read().decode('utf-8'))                            #resp.read()返回byte类型，用decode('utf-8')进行解码


## 带cookie请求头 人人网实践

### 手动添加登录cookie至headers

In [ ]:
# 人人网 - 手动加上登陆过后的cookie，很方便，但是cookie有有效期，过了就没有用了
url='http://www.renren.com/708882071/profile'

headers={
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"   
    ,"cookie": "anonymid=kbvfvse2upxo76; _r01_=1; taihe_bi_sdk_uid=616d1d4b10ead4549dff3231773df9b4; taihe_bi_sdk_session=68ec273f52916b734b3b37bc8bbfeaf8; ick_login=36c92b40-3ba4-4c2c-a674-de9f5a6069e7; _de=3F293AEC34FEF429196E82EB2EB7F3536DEBB8C2103DE356; first_login_flag=1; ln_uact=1977663520@qq.com; ln_hurl=http://hdn.xnimg.cn/photos/hdn121/20141129/1805/h_main_xDgE_27dd00077f3b1986.jpg; id=403872459; ver=7.0; JSESSIONID=abcjrfRcTh4KTbFte5Tlx; jebe_key=3773a40e-4bf0-4bdf-85c7-9626b238eb0a%7Ce1f2d037ce52250e7d6df2a41bcf50a2%7C1593129191091%7C1%7C1593129191180; XNESSESSIONID=53132ec6a5aa; depovince=GW; jebecookies=40a7521c-b656-4346-b2ab-74cbcdc48863|||||; p=79d647178233a5f2b50170847b39a2f79; t=860f8b3595244321efd267c682b838a59; societyguester=860f8b3595244321efd267c682b838a59; ap=403872459; xnsid=e7fb6d4a; loginfrom=null; wp_fold=0"
}

req=request.Request(url,headers=headers) # 用request.Request(url,headers=headers)创建带请求头的url
resp=request.urlopen(req)                # 用request.urlopen(req)发送请求

#写入文件看看长什么样子
with open('renren手动cookie.html','w',encoding='utf-8') as fp:     # write必须写入一个str的数据类型
    fp.write(resp.read().decode('utf-8')) # resp.read()返回的是一个byte的数据类型，所以要decode解码 


### ***使用cookiejar获取动态cookie ***

In [183]:
from urllib import request,parse
from http.cookiejar import CookieJar


# 1. 登录，获取cookie
def get_opener():
    cookiejar=CookieJar()
    handler=request.HTTPCookieProcessor(cookiejar)
    opener=request.build_opener(handler)
    return opener

def login_renren(opener):
    headers={
        "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"   
        }
    data = {
        'email':"1977663520@qq.com"
        ,'password':'62680720sjl'
    }

    login_url='http://www.renren.com/PLogin.do' #使用data登录，获取cookie
    req=request.Request(login_url,data=parse.urlencode(data).encode('utf-8'),headers=headers)# data编码成unicode，再用utf-8编码  
    opener.open(req)                            #只要open记录过cookie就可以了，不需要返回任何东西

    
# 2. 使用带有cookie的opener访问个人主页
def visit_profile(opener):
    profile_url='http://www.renren.com/708882071/profile' # 获取个人主页的时候，不要再新建一个opener，应该使用之前的opener，
    resp=opener.open(profile_url)    # 因为之前的opener已经包含了登录需要的cookie  
                                     
    with open('renrenCookieJar.html','w',encoding='utf-8') as fp:
        fp.write(resp.read().decode('utf-8'))

        
# 3. call functions
if __name__=='__main__':
    opener=get_opener()
    login_renren(opener)
    visit_profile(opener)
    

### 保存cookie到本地

保存cookie到本地，可以用cookiejar的save方法，并且需要指定一个文件名

In [148]:
from urllib import request
from http.cookiejar import MozillaCookieJar

cookiejar=MozillaCookieJar('cookie.txt') # 指定文件名
cookiejar.load(ignore_discard=True) #可以加载调用
handler = request.HTTPCookieProcessor(cookiejar)
opener=request.build_opener(handler)

headers={
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"   
    }
data = {
    'email':"1977663520@qq.com"
    ,'password':'62680720sjl'
}

login_url='http://www.renren.com/PLogin.do' #使用data登录，获取cookie
req=request.Request(login_url,data=parse.urlencode(data).encode('utf-8'),headers=headers)# data编码成unicode，再用utf-8编码  
opener.open(req) 

for cookie in cookiejar:
    print(cookie)

cookiejar.save(ignore_discard=True,ignore_expires=True) # 这样过期的cookie信息也会被保存起来

<Cookie _de=3F293AEC34FEF429196E82EB2EB7F3536DEBB8C2103DE356 for .renren.com/>
<Cookie anonymid=kcckdato-333lyj for .renren.com/>
<Cookie first_login_flag=1 for .renren.com/>
<Cookie id=403872459 for .renren.com/>
<Cookie ln_hurl=http://hdn.xnimg.cn/photos/hdn121/20141129/1805/h_main_xDgE_27dd00077f3b1986.jpg for .renren.com/>
<Cookie ln_uact=1977663520@qq.com for .renren.com/>
<Cookie loginfrom=null for .renren.com/>
<Cookie p=878eeab7b09dbd6425608364625ec2d39 for .renren.com/>
<Cookie societyguester=17177655614415df8b72df8474207afd9 for .renren.com/>
<Cookie t=17177655614415df8b72df8474207afd9 for .renren.com/>
<Cookie ver=7.0 for .renren.com/>
<Cookie xnsid=d45c6c14 for .renren.com/>
<Cookie t=f838128fe2ea8e35cba9d2b3ab4b261f for .renren.com/xtalk/>
<Cookie JSESSIONID=abcM917b9xyy8dYj4MRmx for www.renren.com/>


## 编码解码

### parse.urlencode( ) 编码

In [21]:
from urllib import parse
params= {'name':'爬虫基础','greet':'hello','age':18} #进行编码,把中文用16进制编码 unicode类型
result=parse.urlencode(params) 
result

'name=%E7%88%AC%E8%99%AB%E5%9F%BA%E7%A1%80&greet=hello&age=18'

### parse_qs( ) 还原编码

In [22]:
from urllib import parse
parse.parse_qs(result) #16进制解码回中文

{'name': ['爬虫基础'], 'greet': ['hello'], 'age': ['18']}

### parse.urlparse( )分割url各个成分

In [33]:
from urllib import parse, request
url='https://baike.baidu.com/item/%E7%8F%A0%E6%B1%9F%E4%B8%89%E8%A7%92%E6%B4%B2/621061'
result1=parse.urlparse(url)
print(result1,result1.netloc)

ParseResult(scheme='https', netloc='baike.baidu.com', path='/item/%E7%8F%A0%E6%B1%9F%E4%B8%89%E8%A7%92%E6%B4%B2/621061', params='', query='', fragment='') baike.baidu.com


## 拉勾网实践 - request 和 selenium webdriver

### 拉勾网实践1 - request 增加http请求头

In [187]:
from urllib import request,parse
url='https://www.lagou.com/jobs/positionAjax.json?px=default&city=%E4%B8%8A%E6%B5%B7&needAddtionalResult=false'

headers={             # 添加请求头 user-agent, referer
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
    ,'referer': 'https://www.lagou.com/jobs/list_Python/p-city_3?px=default'
    ,'cookie': 'JSESSIONID=ABAAAECABFAACEA3D60243F733ADE57004CF5045C58206D; WEBTJ-ID=07012020%2C224523-1730e0ee1c7335-04bf4c280ccb1-31627403-2073600-1730e0ee1c84a9; RECOMMEND_TIP=true; _ga=GA1.2.515260754.1593668725; user_trace_token=20200702134525-3dc907fb-cc8c-493a-90be-4acae9a1eff5; LGUID=20200702134525-630d15d9-7779-4cf3-a0d6-c6fae3fc761f; index_location_city=%E5%85%A8%E5%9B%BD; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%221730e1055a1236-03e6e2e1cec109-31627403-2073600-1730e1055a3690%22%2C%22%24device_id%22%3A%221730e1055a1236-03e6e2e1cec109-31627403-2073600-1730e1055a3690%22%7D; _gid=GA1.2.1358182150.1594180320; PRE_UTM=; PRE_LAND=https%3A%2F%2Fwww.lagou.com%2F; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1593668725,1594180321; LGSID=20200708115201-90705303-7ff4-4e9c-ba35-46c874dbdc21; PRE_HOST=www.google.com; PRE_SITE=https%3A%2F%2Fwww.google.com%2F; TG-TRACK-CODE=index_navigation; _gat=1; X_HTTP_TOKEN=9a495d5bd3a415de11408149514c782f8cac9fa179; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1594180412; LGRID=20200708115332-dfb531af-c0f3-47d8-81d6-547694353b05; SEARCH_ID=b476835face34153998b198d9875d034'

}

data={                # POST 方法需要携带data信息, 并且需要在请求头里表明method
    'first':'true',   # data一般是需要编码，urlencode是unicode，最后需要编码utf-8
    'pn':'1',
    'kd':'Python'
}
req=request.Request(url,headers=headers,data=parse.urlencode(data).encode('utf-8'),method='POST') #给这个URL增加了请求头，返回req（带头的url）
resp=request.urlopen(req) # urlopen打开这个req
resp.read().decode('utf-8')


'{"status":false,"msg":"您操作太频繁,请稍后再访问","clientIp":"162.235.252.204","state":2402}\n'

### 拉勾网实践2 - webdriver 模拟浏览器

In [56]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import os

url='https://www.lagou.com/jobs/list_python%E7%88%AC%E8%99%AB/p-city_3?px=default&gx=%E5%85%A8%E8%81%8C&gj=&xl=%E7%A1%95%E5%A3%AB&isSchoolJob=1&district=%E9%95%BF%E5%AE%81%E5%8C%BA#filterBox'
chromePath=os.path.join('/Users/jialinshi/Desktop', 'chromedriver')

browser=webdriver.Chrome(executable_path=chromePath)   #打开模拟浏览器
browser.maximize_window()

browser.get(url)                              #模拟浏览器拿到html代码，用css selector定位信息爬取
sleep(3)
html=browser.page_source
soup=BeautifulSoup(html,'html.parser')
sleep(3)

def info(category,css_selector):              #爬取表格这类高度相似的的信息可以写一个def
    category=[]
    for x in soup.select(css_selector):
        category.append(x.text)
    return category

data=pd.DataFrame()

# call function
data['salary']=info('salary','.money')
data['position']=info('position','h3')
data['company']=info('company','.company_name a')
data['industry']=[x.strip().split('\n') for x in info('industry','.industry')] # 字符串去空格去换行\n
data['tag']=[x.strip().split('\n') for x in info('tag','.list_item_bot .li_b_l')]
data

In [94]:
data

,salary,position,company,industry,tag
0,20k-40k,仿真工程师,智加科技,[人工智能 / A轮 / 50-150人],"[人工智能, 建模, 计算机视觉]"
1,30k-50k,运动控制工程师,智加科技,[人工智能 / A轮 / 50-150人],"[人工智能, 算法]"
2,20k-35k,机器视觉、神经网络和深度学习算法研发工程师与机器人,上海岭先机器人科技股份有限公司,[人工智能 / 未融资 / 15-50人],"[人工智能, 计算机视觉, 算法, 机器学习]"
3,30k-60k,算法工程师,智加科技,[人工智能 / A轮 / 50-150人],"[汽车, 地图, 人工智能, 算法, 机器学习]"


# requests库

## get/post 请求

In [200]:
#get请求
import requests
url='https://www.baidu.com/'
response=requests.get(url)

print(type(response.text)) #返回str格式，.text方法会自动解码，猜测解码方法,可能不准确，出现乱码
print(type(response.content))#返回byte数据类型，不解码 
print(response.url)

with open('baidu.html','w',encoding='utf-8') as fp:
    fp.write(response.content.decode('utf-8')) #fp.write()必须传入一个str数据类型，所以content必须解码

<class 'str'>
<class 'bytes'>
https://www.baidu.com/


In [201]:
#post请求
import requests
url='https://www.lagou.com/jobs/positionAjax.json?px=default&city=%E4%B8%8A%E6%B5%B7&needAddtionalResult=false'

data={
    "first": "true"
    ,"pn": "1"
    ,"kd": "Python"
}
headers={             # 手动添加cookies
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
    ,'referer': 'https://www.lagou.com/jobs/list_Python/p-city_3?px=default'
    ,'cookie': 'JSESSIONID=ABAAAECABFAACEA3D60243F733ADE57004CF5045C58206D; WEBTJ-ID=07012020%2C224523-1730e0ee1c7335-04bf4c280ccb1-31627403-2073600-1730e0ee1c84a9; RECOMMEND_TIP=true; _ga=GA1.2.515260754.1593668725; user_trace_token=20200702134525-3dc907fb-cc8c-493a-90be-4acae9a1eff5; LGUID=20200702134525-630d15d9-7779-4cf3-a0d6-c6fae3fc761f; index_location_city=%E5%85%A8%E5%9B%BD; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%221730e1055a1236-03e6e2e1cec109-31627403-2073600-1730e1055a3690%22%2C%22%24device_id%22%3A%221730e1055a1236-03e6e2e1cec109-31627403-2073600-1730e1055a3690%22%7D; _gid=GA1.2.1358182150.1594180320; PRE_UTM=; PRE_LAND=https%3A%2F%2Fwww.lagou.com%2F; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1593668725,1594180321; LGSID=20200708115201-90705303-7ff4-4e9c-ba35-46c874dbdc21; PRE_HOST=www.google.com; PRE_SITE=https%3A%2F%2Fwww.google.com%2F; TG-TRACK-CODE=index_navigation; _gat=1; X_HTTP_TOKEN=9a495d5bd3a415de11408149514c782f8cac9fa179; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1594180412; LGRID=20200708115332-dfb531af-c0f3-47d8-81d6-547694353b05; SEARCH_ID=b476835face34153998b198d9875d034'
}
response=requests.post(url,data=data,headers=headers) # Post请求需要指定data
response.json()

{'status': False,
 'msg': '您操作太频繁,请稍后再访问',
 'clientIp': '162.235.252.204',
 'state': 2402}

## 代理ip处理器

In [ ]:
import requests
url='http://httpbin.org/ip'

headers={             
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

proxy={                           #可以找免费代理
    'http':'171.14.209.180:27829'
}

resp=requests.get(url,headers=headers,proxies=proxy)

with open('xx.html','w',encoding='utf-8')as fp:
    fp.write(resp.text)

## cookie

In [208]:
import requests

url='http://www.renren.com/PLogin.do'

headers={
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"   
    }
data = {
    'email':"1977663520@qq.com"
    ,'password':'62680720sjl'
    }

response=requests.get(url,data=data,headers=headers)
response.cookies.get_dict()


{'_r01_': '1',
 'depovince': 'GW',
 'jebecookies': '5b2f08e0-d729-435c-bcb5-dd294e55b373|||||',
 'JSESSIONID': 'abcBONYoH35N54GHV1Smx'}

## session （相当于request的opener）

In [203]:
import requests

login_url='http://www.renren.com/PLogin.do'
profile_url='http://www.renren.com/708882071/profile'

headers={
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"   
    }
data = {
    'email':"1977663520@qq.com"
    ,'password':'62680720sjl'
    }

#登陆
session=requests.session()
session.post(login_url,data=data,headers=headers)


#访问个人主页
response=session.get(profile_url)

with open('renrenSession.html','w',encoding='utf-8') as fp:
    fp.write(response.content.decode('utf-8'))

In [ ]:
笔记：
        1. request中的opener可以发送多个请求，请求之间可以共享opener
        2. requests中可以使用session对象来共享cookie
        3. 用session.post()方法登陆，然后用同一个session去get新的url